<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex07/Pedro%20Gengo/Pedro_Gengo_Aula_7_Exerc%C3%ADcio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Pedro Gabriel Gengo Lourenco'
print(f'Meu nome é {nome}')

Meu nome é Pedro Gabriel Gengo Lourenco


#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural simples para prever a proxima palavra de um texto, dada as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Língua".

Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [8]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

     |████████████████████████████████| 4.2 MB 29.7 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 6.6 MB 65.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [4]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [5]:
# Check which GPU we are using
!nvidia-smi

Tue May 17 13:50:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [7]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokenizer = tokenizer
        self.context_size = context_size
        self.x, self.y = self._split_tokens_in_contextualized_windows(texts)

    def _split_tokens_in_contextualized_windows(self, texts):
      x, y = [], []
      for t in texts:
        tokenized = tokenize(t, self.tokenizer)
        if len(tokenized) > self.context_size:
          for i in range(len(tokenized) - self.context_size):
              x.append(tokenized[i: i + self.context_size])
              y.append(tokenized[i + self.context_size])
      return torch.LongTensor(x), torch.LongTensor(y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

## Teste se sua implementação do MyDataset está correta

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [9]:
dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

### Debuggando o tokenizer

In [10]:
tokenizer.convert_ids_to_tokens(tokenize(dummy_texts[0], tokenizer)), tokenizer.convert_ids_to_tokens(tokenize(dummy_texts[1], tokenizer))

(['Eu', 'gosto', 'de', 'correr'],
 ['Ela', 'gosta', 'muito', 'de', 'comer', 'pi', '##zza'])

No resultado da célula acima podemos ver os tokens gerados pelo tokenizador. Assim, podemos montar o dataset esperado, sendo:

| x | y |
|---|---|
|[Eu, gosto, de]|correr|
|[Ela, gosta, muito]|de|
|[gosta, muito, de]|comer|
|[muito, de, comer]|pi|
|[de, comer, pi]|##zza|

Sendo assim, esperamos que o dataset tenha um tamanho de 5.

### Testes

In [11]:
dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [12]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [13]:
words_per_line = [len(line.split()) for line in open('sample_brwac.txt').readlines()]

In [14]:
sum(words_per_line) / len(words_per_line)

788.32044

In [15]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

In [16]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [17]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim, hidden_size):
        """
        Implements the Neural Language Model proposed by Bengio et al."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        super().__init__()
        self.emb = torch.nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = torch.nn.Linear(embedding_dim * context_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, vocab_size, bias = False)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        """
        output = self.emb(inputs)
        output = output.view(inputs.shape[0], -1)
        output = torch.nn.functional.relu(self.linear1(output))
        output = self.linear2(output)
        return output

## Teste o modelo com um exemplo

In [18]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
    hidden_size=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [19]:
tokenizer.vocab_size

29794

In [20]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5794304


## Assert da Perplexidade


In [21]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    pred_probs = torch.nn.functional.softmax(logits, dim=1)
    valid_probs = pred_probs.gather(1, target.view(-1, 1))
    cross_entropy = -torch.mean(torch.log(valid_probs))
    return torch.exp(cross_entropy)

n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              30087
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [32]:
max_examples = 200_000_000
eval_every_steps = 5000
lr = 3e-5
best_ppl = 1e5  # Para evitar usar um modelo overfittado, salvo o modelo com melhor ppl para usar depois


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=512, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=512)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device)) * 0.9
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss / 0.9)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl < best_ppl:
                  best_ppl = valid_ppl
                  torch.save(model.state_dict(), 'best_model.pt')

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30359.69, valid ppl: 30743.96
5000 steps; 2560000 examples so far; train ppl: 2435.19, valid ppl: 1539.77
10000 steps; 5120000 examples so far; train ppl: 1351.66, valid ppl: 1209.86
15000 steps; 7680000 examples so far; train ppl: 1102.09, valid ppl: 1007.55
20000 steps; 10240000 examples so far; train ppl: 933.42, valid ppl: 871.81
25000 steps; 12800000 examples so far; train ppl: 817.94, valid ppl: 776.74
30000 steps; 15360000 examples so far; train ppl: 735.60, valid ppl: 705.34
35000 steps; 17920000 examples so far; train ppl: 673.78, valid ppl: 648.40
40000 steps; 20480000 examples so far; train ppl: 622.89, valid ppl: 601.81
45000 steps; 23040000 examples so far; train ppl: 582.06, valid ppl: 562.92
50000 steps; 25600000 examples so far; train ppl: 543.20, valid ppl: 529.00
55000 steps; 28160000 examples so far; train ppl: 511.93, valid ppl: 499.41
60000 steps; 30720000 examples so far; train ppl: 474.64, valid ppl: 473.77
65000 steps; 3328

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [33]:
model = LanguageModel(
            vocab_size=tokenizer.vocab_size,
            context_size=context_size,
            embedding_dim=128,
            hidden_size=256,
        ).to(device)
model.load_state_dict(torch.load('best_model.pt'))
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 182.5328581355589


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [34]:
prompt = 'Eu gosto de comer pizza pois me faz'  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com que
Eu gosto de comer pizza pois me faz com que o
Eu gosto de comer pizza pois me faz com que o que
Eu gosto de comer pizza pois me faz com que o que eu
Eu gosto de comer pizza pois me faz com que o que eu não
Eu gosto de comer pizza pois me faz com que o que eu não tinha
Eu gosto de comer pizza pois me faz com que o que eu não tinha mais
Eu gosto de comer pizza pois me faz com que o que eu não tinha mais nada
Eu gosto de comer pizza pois me faz com que o que eu não tinha mais nada.


In [35]:
prompt = 'Se os carros são como as lanchas, as motos são'  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Se os carros são como as lanchas, as motos são os
Se os carros são como as lanchas, as motos são os que
Se os carros são como as lanchas, as motos são os que estão
Se os carros são como as lanchas, as motos são os que estão em
Se os carros são como as lanchas, as motos são os que estão em um
Se os carros são como as lanchas, as motos são os que estão em um país
Se os carros são como as lanchas, as motos são os que estão em um país,
Se os carros são como as lanchas, as motos são os que estão em um país, que
Se os carros são como as lanchas, as motos são os que estão em um país, que não
Se os carros são como as lanchas, as motos são os que estão em um país, que não se


In [36]:
prompt = 'Foi muito bom te ver, mas já está na hora de'  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Foi muito bom te ver, mas já está na hora de fazer
Foi muito bom te ver, mas já está na hora de fazer o
Foi muito bom te ver, mas já está na hora de fazer o que
Foi muito bom te ver, mas já está na hora de fazer o que eu
Foi muito bom te ver, mas já está na hora de fazer o que eu não
Foi muito bom te ver, mas já está na hora de fazer o que eu não tinha
Foi muito bom te ver, mas já está na hora de fazer o que eu não tinha mais
Foi muito bom te ver, mas já está na hora de fazer o que eu não tinha mais nada
Foi muito bom te ver, mas já está na hora de fazer o que eu não tinha mais nada.
Foi muito bom te ver, mas já está na hora de fazer o que eu não tinha mais nada..


In [38]:
prompt = 'Nas minhas próximas férias quer ir viajar para'  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Nas minhas próximas férias quer ir viajar para o
Nas minhas próximas férias quer ir viajar para o seu
Nas minhas próximas férias quer ir viajar para o seu trabalho
Nas minhas próximas férias quer ir viajar para o seu trabalho.
Nas minhas próximas férias quer ir viajar para o seu trabalho. O
Nas minhas próximas férias quer ir viajar para o seu trabalho. O que
Nas minhas próximas férias quer ir viajar para o seu trabalho. O que é
Nas minhas próximas férias quer ir viajar para o seu trabalho. O que é o
Nas minhas próximas férias quer ir viajar para o seu trabalho. O que é o caso
Nas minhas próximas férias quer ir viajar para o seu trabalho. O que é o caso de
